# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是《易经》中的一个卦象，由两个阴爻夹一个阳爻构成，象征着天、阳气、刚强等含义。乾卦的卦辞为：“乾，元亨利贞。”意味着“乾”是宇宙万物的根本原理，具有活动和创造的力量。

乾卦具有以下特点：

1. 阳爻在中间：乾卦的特点是阳爻位于卦象中心，表现了阳刚之气的特性。

2. 两个阴爻夹一个阳爻：乾卦是由两个阴爻夹一个阳爻构成，这种排列象征着天、阳气、刚强等含义。

3. 刚强有力：乾卦所象征的阳刚之气是强烈、刚毅的，具有克服困难、带领团队前进的力量。

4. 正义、诚实：乾卦还象征着正义、诚实、公正等品质，表明在处理事物时要坚守正道，秉持诚实守信的原则。

5. 发展、进步：乾卦所代表的刚强力量是事物发展的动力，它能推动事物不断前进，走向繁荣发展。

总之，乾卦是《易经》中一个象征阳刚、力量、正义和发展的卦象。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的一个卦象，由两个阳爻夹一个阴爻构成。这个卦象象征着诉讼、争端、矛盾和困扰等。以下是讼卦的卦辞和象征意义：

卦辞：讼，元吉，利见大人。

 meaning: "讼"，表示诉讼、争端和困扰；"元吉"，意为原始的、正当的吉祥；"利见大人"，意味着在争端中可以得到有实力的人的支持和帮助。

特点：

1. 阳爻在中间：讼卦的特点是阳爻位于卦象中心，表现了阳刚之气的特性。

2. 两个阳爻夹一个阴爻：讼卦是由两个阳爻夹一个阴爻构成，这种排列象征着处理争端和诉讼时需要阳刚之气的支持。

3. 矛盾和困扰：讼卦所象征的矛盾和困扰表示在处理事物时可能会遇到困难和阻力，需要有坚定的信念和实力来解决。

4. 诉讼和争端：讼卦象征着诉讼和争端，表明在处理问题时可能会面临法律诉讼等争端。

5. 需要和解：讼卦也表明，在面对矛盾和困扰时，需要寻求和解和妥协，以便解决问题。

总之，讼卦是《易经》中一个象征诉讼、争端、矛盾和困扰的卦象。在面对问题时，我们需要坚定信念，寻求和解和妥协，以解决问题。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250820_084743"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由两个乾卦叠加而成，代表天行健。在周易中，这是一个吉卦，象征着天行健健，君子擬法。乾卦代表阳，代表力量、刚健、刚毅、刚健、刚毅、正派、刚健、求进、勇敢、坚定，象征着大吉大利。乾卦的核心哲学是：天行健，君子拟法。大吉，利市如春起之苗。大吉，有利于进举，可求得完美的成功。大吉，可以震动他人，可以完成大事。大吉，可以得到君子之喜，可以震动周围的人。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250820_084743）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250820_084743）：
[gMASK]sop 周易中的讼卦是什么样子? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '占卜结果的一种。师卦是由坎卦（水）和坤卦（地）相叠而成，象征着军队的力量和军情的总指挥。在这个卦象中，预示着吉祥无灾。君子观此卦象，应当以仁爱、宽厚、正直的品德来行事，必能获得成功。', 'content': '\n师卦详情：\n\n1. 初六，阳ik，表示刚强坚毅，刚中有柔，自信满盈，无患难，此乃凶中藏吉之象。\n2. 初七，阳i，充满自信，但过于骄傲，导致吉凶难辨。\n3. 官动，阳i，表示官方支持，适用于官场、官事、官运等方面，需谨慎处理，可获得吉祥。\n4. 初五，阳i，刚中有柔，表示坚韧坚定，有耐心和毅力。\n5. 初四，阳i，表示坚毅果断，充满自信，可获得正运。\n\n师卦的解释，并非完全凶险，而是有凶有吉，需要坚强坚毅，灵活机动，刚中有柔，耐心坚持，才能化凶为吉。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250820_084743）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，含有元、始、长、高、广等特性。它由两个卦象组成，上卦是乾，下卦是坤。上卦的六条线（即爻）都是阳线，下卦的六条线都是阴线，这表示了天阳地阴的特性。

在周易中，乾卦代表着天，象征着刚健、强劲、积极、勇敢的品质。它代表着刚强不屈不挠的精神，象征着锐利、锐利的特性，同时代表着广大、广博的视野。

在周易的解卦中，乾卦表示天圆，地平方，刚强而强劲，代表着锐利、刚强的特性。在卜筮时，预示着大吉大利，同时预示着先后的顺利。

在解卦时，乾卦预示着大吉大利，并带来顺利的前程，同时预示着先后的顺利。在卜筮时，意味着指数非常吉祥，预示着地面平旷无垠，预示着先后的行程平安。

在卜筮时，指数非常吉祥，预示着吉利的指数非常高，因此，在周易中，乾卦意味着大吉大利。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 

 乾卦是周易中的一卦，代表天，象征着刚健强劲的特性。它由六个阳爻组成，代表着积极、主动、进取的态度。在周易中，乾卦预示着大吉大利，事业如日中天。同时，乾卦也象征着刚强坚毅的品质，以及长久的持续力。

乾卦的卦辞为：“元、亨、利、贞”，元表示开始，亨表示顺利，利表示吉祥，贞表示正，表示事业如日中天，吉祥如意。在卜卦时，顺随天时地利，刚强坚毅的特性有利于事业成功。

在事业和经商方面，刚强坚毅的乾卦有利于积极进取，有利于实现目标。在人际关系方面，刚强坚毅的特性可能使对方感到威胁，因此需要谨慎对待，尊重他人，有利于建立良好的人际关系。

在生活方面，刚强坚毅的特性有利于解决困难，但需要谨慎对待，尊重他人。在决策方面，应该坚持原则，谨慎决策，有利于实现目标。

总之，乾卦象征着刚健强劲的特性，预示着大吉大利，事业如日中天，但需要谨慎对待，尊重他人。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是易经中的一卦，由卦象观之，上卦为坤，代表地，代表坤德，含育万物，承载大地球；下卦为坎，代表水，代表坎德，游跃于地平。

地水师卦代表地水相交之象，象征着耐心等待，深入挖掘潜力，时机成熟即可行动。同时，卦象也预示著：

1. 事情会发生变化，可能出现转机；
2. 需要灵活变通，以适应变化的局势；
3. 需要坚持正道，防止陷入水中深潭。

综合来看，地水师卦意味着耐心、谨慎地面对变化，抓住时机，以变通求进。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表地上的水。根据《象辞》，这一卦象被解释为“地上的水”，象征着地上的水景。根据《断易天机》，只有具备高洁品德的人，才能在这样的环境中得到伸展，从而获得吉祥。


根据北宋易学家邵雍的解卦，地水师卦具有以下含义：

初六：地水师卦。

六二：地水师卦。

六三：地水师卦。

六四：地水师卦。

六五：地水师卦。

上六：地水师卦。

根据《象辞》，初六、六二、六三、六四、六五、上六的卦象分别代表天象中的消息、地上的水、地上的水、地上的水、地上的水、地上的水，以及地上的水，地上的水，分别代表大地上的景物。

根据北宋易学家邵雍的解卦，地水师卦表示：

初六：地水师卦。

六二：地水师卦。

六三：地水师卦。

六四：地水师卦。

六五：地水师卦。

上六：地水师卦。

根据《断易天机》，地水师卦的运势解释为：

初六：初入世间，面临艰难，需要坚持道德，以得到吉祥。

六二：运程稳定，得贵人相助，可以顺利发展。

六三：面临困境，需要坚持道德，以得到吉祥。

六四：运程稳定，需要努力不懈，以得到吉祥。

六五：运势顺利，得贵人相助，可以顺利发展。

上六：运势顺利，得贵人相助，可以顺利发展。

根据《象辞》，地水师卦预示着：

初六：初入世间，面临艰难，需要坚持道德，以得到吉祥。

六二：运程稳定，得贵人相助，可以顺利发展。

六三：面临困境，需要坚持道德，以得到吉祥。

六四：运程稳定，需要努力不懈，以得到吉祥。



In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易卦象中的一枚卦，由上卦坤（地）和下卦乾（天）组成，预示着利害相参参的卦象。在卜筮时， Collective () 
讼卦的象辞为：利见大人。 ultimate() 
 讼卦表示为大象利见大人，有利于解决争端，但是最终会得到好处。因此，讼卦的意义为：

初六：云风交驰，利见大人，象 Crowds（众）

七七：利见大人，而往上学。的人群中，利见大人，得到好处。

七二：利见大人，往者则安，疑重。 

七三：利见大人，而往者疑，远行。 

七四：利见大人，而往者疑，疑重。 

七五：利见大人，而往者利，得到好处。

七六：利见大人，而往者利，疑重。 

七七：利见大人，而往者疑，得志。

讼卦的时势为初时，和创新为利见大人，高台为往者疑，疑重，远行。 

讼卦的经商策略为暂且等待，不要冒险，避免发生意外，稳定收益。 

讼卦的结局为高台之天，利见大人，利得重，利见大人，有利于高台。 

讼卦预示着利害相参参的卦象，有利于解决争端，但是最终会得到好处。因此，讼卦意味着在处理争端时需要谨慎、温和、高台，并且高台之天，利见大人，利得重，利见大人。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易中的一卦，代表诉讼、争斗和矛盾卦象。它由两个卦象组成，上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争端的产生。讼卦中有利有弊，利在天时，利在德，有利有弊，需要善于权衡。在卜卦时， initial卦象为天时，利在德，最终预测顺利，但需谨慎。

讼卦的核心哲学是：天时有时利，有时不利，需要人们谨慎行动。在处理争端和诉讼时，初始可能会顺利，但随着时间的推移，可能会出现延误和困难。因此，在处理诉讼时，初始可能顺利，但最终可能会遇到挫折，需要人们谨慎对待。

讼卦的经商指引是：初时顺利，但后来可能会遇到困难和挫折。因此，在商业谈判中，需要谨慎对待，权衡利弊，寻求公平。

讼卦的决策提示是：面对争端和诉讼，要寻求和解，避免冲突。在冲突中，要冷静、谨慎，避免过度争执，以保持和平。

讼卦的运势提示是：初时顺利，但后来可能会遇到困难和挫折。因此，在寻求成功时，需要保持冷静、谨慎，避免过度自信，以保持稳定和繁荣

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 50
timestamp = "20250820_091435"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的一种卦象，它是由两个离卦（火）在上，一个震卦（雷）在下的卦象，因此乾卦也可以看作是一种阳刚之卦。乾卦象征着天，代表着刚强、强大、积极向上、积极进取、顺从规律，也象征着一种完成和成熟。

乾卦的卦象是由三个阴线和六个阳线组成的，其中阴线表示空，阳线表示实，完成和充实。乾卦的卦象为：

* 初九：夕夕其鸣，闻丧其耦。
* 初六：鸡鸣，言笑， watch
* 是以：鼓瑟，以丧其耦。
*  九二：92 刚健，  Temperance
* 九五：飞黄腾达，家宅 suf
* 上九：天命有限，人道终

乾卦的卦辞：元，亨，利，贞。

象辞：天

乾卦的卦辞描述了一个人在世间行事的态度和准则，它告诉我们，要在生活中保持刚强和进取，才能获得成功和幸福。

乾卦的卦辞也表达了人们对于天的敬畏和尊重，它告诉我们，人应该顺从天理，遵循自然规律，才能获得长久的幸福。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是乾为天，乾为元的上市。乾卦是乾卦的第一卦，它象征着刚强，独立，积极，积极，向阳，积极进取。乾卦是阳刚之卦，象征着天，也可以象征人。它反映了人们向上，向善，积极进取的精神。
 乾卦的卦象由两个阴爻夹一个阳爻构成，阴爻表示柔弱，阳爻表示刚强，它们相互配合，形成刚强独立的精神。乾卦的卦象也可以看作是一个阳爻在两个阴爻之间，表示天，也可以表示人，表明了人们向上，向善，积极进取的精神。


In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦（噬渊卦、师卦）是《易经》中的一种卦象，卦象为：上艮下坤。象征师兵，师兵众多，正义之师。

师卦象征师众，师卦是根据坤卦和艮卦的组合而得，坤卦代表地，艮卦代表山。师卦象征山多，师从山出，象征军队众多，正义之师。

师卦的卦象为：上坤下艮。坤代表地，象征万物，师的合法性来源于“万物”，艮代表山，象征师兵众多。

师卦的卦辞：师，元，喜。

初爻：贞，吉，悔。

二爻：吉，无咎。

三爻：凶，吉。

四爻：吉，悔。

师卦卦辞：正义之师，无咎，多多 effort。

附：师卦原文：师。贞，元，喜。初爻：贞，吉，悔。二爻：吉，无咎。三爻：凶，吉。四爻：吉，悔。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [21]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（乾卦下艮卦上）卦辞：天与地讼，吉。无悔。象曰：天行健，君子以朋友志。

白话文：讼卦的象征是天与地相争，大吉。意为鸡肉美酒，无恶。卦辞：上艮下乾，为讼卦。象征地为争斗，象征天行健，君子以志。

 adj. 诉讼

解：讼卦象征争斗,但也有成功的时候。诉冤获理,吉兆。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
